In [1]:
import numpy as np
import sys
import os
import matplotlib.pyplot as plt
import copy
import torch as tr

# import scripts
from src.bayes_opt import bayesian_optimiser
from src.api_helper import api_utils
from src.api_helper import env
from datetime import datetime as dt, date as dt_date

device = tr.device("cuda" if tr.cuda.is_available() else "cpu")
dtype = tr.float32

##### data

In [2]:
@api_utils.multi_thread_transform
def api(query): return env.rosenbrock(query)  # computation on cpu

x0, y0 = tr.rand(2).view(1, -1), tr.tensor([[-1]], dtype=dtype).view(-1, 1)  
r0 = env.rosenbrock(x0)  # initial reward on which normalisation based

x0, y0= x0.to(device), y0.to(device)  # data in GPU

print("init query", x0); print(f"init reward {api(x0, r0, device)}, origin reward: {r0.item():.2f}")  # x,y stay on GPU while r0 in cpu

init query tensor([[0.2532, 0.7139]])
init reward tensor([[-1.]]), origin reward: 42.78


##### OPT

In [3]:
# hyperparameters
T = 10  # total number of iterations

# gp; includes "MA2.5", "SE", "RQ", "LR", "PO"
gp_name, gp_params = "MA2.5",{
                          "mode": "raw",      # "raw", "add", "pro" for GPs
                          "opt":"ADAM",  # opt for MLE; (quasi_newton, ADAM)
                          "epochs":128,       # epoch to run, if chosen ADAM
                          "lr":1e-1,          # learning rate for ADAM
                         }
# q-parallelism (if use analytical acq_func, q must be 1)
batch_size = 2

acq_params = { 
    "acq_name" : "qUCB",          # acqu func; includes: "EI", "UCB", "qEI", "qUCB", "qKG"
    "N_start": 32,               # number of starts for multi-start SGA
    "raw_samples" :512,          # heuristic initialisation 
    "N_MC_sample" : 256,         # number of samples for Monte Carlo simulation
    "num_fantasies": 128,        # number of fantasies used by KG
    "beta":1.,                   # used by UCB/qUCB
               }

bayes_opt = bayesian_optimiser(gp_name, gp_params, device, acq_params)

In [4]:
%%time
xs, ys = bayes_opt.outer_loop(T, (-4,4),x0, y0, r0, api, batch_size)  # maximising reward

NLML: -3.57
Iter: 1, reward: -59.11
NLML: 9,184,611.00
Iter: 2, reward: -1.21
NLML: 121,995.85
Iter: 3, reward: -0.41
NLML: 14,971.40
Iter: 4, reward: -0.20
NLML: 3,935.44
Iter: 5, reward: -0.39
NLML: 1,695.76
Iter: 6, reward: -0.10
NLML: 920.81
Iter: 7, reward: -0.18
NLML: 574.01
Iter: 8, reward: -0.02
NLML: 455.75
Iter: 9, reward: -0.05
NLML: 331.16
Iter: 10, reward: -0.01
CPU times: user 12.8 s, sys: 24.6 s, total: 37.4 s
Wall time: 9.53 s


In [16]:
xs.shape

torch.Size([41, 2])